# 🚀 **Módulo 4: Treinando como um personal trainer**

## **Aula 4.1: Configuração do treinamento**

---

### **Tá, mas como treinar a IA?**

Imagine que você é um personal trainer e vai treinar um atleta. Você precisa:
- ✅ Definir o objetivo (força, resistência, velocidade)
- ✅ Criar um plano de treino
- ✅ Acompanhar o progresso
- ✅ Ajustar quando necessário

**Fine Tuning é a mesma coisa!** Vamos treinar nossa IA para ser especialista no que você precisa.

**Por que o treinamento é crucial?**

É como ensinar um funcionário:
- **Treino ruim**: Funcionário aprende coisas erradas
- **Treino bom**: Funcionário vira especialista

---

**🖼️ Sugestão de imagem**: Personal trainer monitorando treino vs configuração de treinamento de IA

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Importações necessárias
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import json
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

## **1. Carregando dados e modelo**

Vamos começar carregando nossos ingredientes:

In [ ]:
# Carregando dataset
print("📊 CARREGANDO DATASET\n")

try:
    with open('datasets/exemplo_vendas.json', 'r', encoding='utf-8') as f:
        dataset_vendas = json.load(f)
    
    print(f"✅ Dataset carregado: {len(dataset_vendas)} exemplos")
    
    # Mostrando exemplo
    print("\n📝 Exemplo do dataset:")
    exemplo = dataset_vendas[0]
    print(f"   Pergunta: {exemplo['messages'][0]['content']}")
    print(f"   Resposta: {exemplo['messages'][1]['content']}")
    
except FileNotFoundError:
    print("⚠️  Dataset não encontrado. Criando exemplo...")
    dataset_vendas = [
        {
            "messages": [
                {"role": "user", "content": "Qual é o melhor iPhone?"},
                {"role": "assistant", "content": "O iPhone 15 Pro Max é o melhor para fotografia e performance."}
            ]
        }
    ]

# Carregando modelo pequeno para demonstração
print("\n🤖 CARREGANDO MODELO\n")

model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"✅ Modelo carregado: {model_name}")
print(f"📊 Parâmetros: {model.num_parameters():,}")

# Configurando tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Tokenizer configurado!")

## **2. Configurando LoRA**

Vamos configurar o LoRA para treinamento eficiente:

In [ ]:
# Configurando LoRA
print("🔧 CONFIGURANDO LORA\n")

# Configuração LoRA
lora_config = LoraConfig(
    r=16,  # Rank da decomposição
    lora_alpha=32,  # Escala dos pesos LoRA
    target_modules=["q_proj", "v_proj"],  # Módulos para aplicar LoRA
    lora_dropout=0.1,  # Dropout para regularização
    bias="none",  # Como tratar bias
    task_type="CAUSAL_LM"  # Tipo de tarefa
)

print("📋 Configuração LoRA:")
print(f"   Rank (r): {lora_config.r}")
print(f"   Alpha: {lora_config.lora_alpha}")
print(f"   Target modules: {lora_config.target_modules}")
print(f"   Dropout: {lora_config.lora_dropout}")

# Aplicando LoRA ao modelo
model = get_peft_model(model, lora_config)

print(f"\n✅ LoRA aplicado!")
print(f"📊 Parâmetros treináveis: {model.print_trainable_parameters()}")

print("\n💡 LoRA permite treinar apenas uma pequena parte do modelo, economizando memória e tempo!")

## **3. Preparando dados para treinamento**

Vamos preparar os dados no formato correto:

In [ ]:
# Função para preparar dados
def preparar_dados_treinamento(dataset, tokenizer, max_length=512):
    """
    Prepara dados para treinamento
    """
    dados_processados = []
    
    for exemplo in dataset:
        # Formatando como conversa
        texto = f"User: {exemplo['messages'][0]['content']}\nAssistant: {exemplo['messages'][1]['content']}"
        
        # Tokenizando
        tokens = tokenizer(
            texto,
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        dados_processados.append({
            "input_ids": tokens["input_ids"].squeeze(),
            "attention_mask": tokens["attention_mask"].squeeze(),
            "labels": tokens["input_ids"].squeeze().clone()
        })
    
    return dados_processados

# Preparando dados
print("🔄 PREPARANDO DADOS PARA TREINAMENTO\n")

dados_treinamento = preparar_dados_treinamento(dataset_vendas, tokenizer)

print(f"✅ Dados preparados: {len(dados_treinamento)} exemplos")

# Criando dataset do Hugging Face
dataset_hf = Dataset.from_list(dados_treinamento)

print(f"📊 Dataset criado: {len(dataset_hf)} exemplos")
print(f"🔧 Colunas: {dataset_hf.column_names}")

# Mostrando exemplo processado
print("\n📝 Exemplo processado:")
exemplo_processado = dataset_hf[0]
print(f"   Input IDs shape: {exemplo_processado['input_ids'].shape}")
print(f"   Attention mask shape: {exemplo_processado['attention_mask'].shape}")
print(f"   Labels shape: {exemplo_processado['labels'].shape}")

## **4. Configurando argumentos de treinamento**

Agora vamos configurar os parâmetros do treinamento:

In [ ]:
# Configurando argumentos de treinamento
print("⚙️ CONFIGURANDO ARGUMENTOS DE TREINAMENTO\n")

training_args = TrainingArguments(
    output_dir="./checkpoints",  # Diretório para salvar checkpoints
    num_train_epochs=3,  # Número de épocas
    per_device_train_batch_size=2,  # Batch size por dispositivo
    gradient_accumulation_steps=4,  # Acumulação de gradientes
    learning_rate=2e-4,  # Taxa de aprendizado
    warmup_steps=100,  # Passos de aquecimento
    logging_steps=10,  # Frequência de logs
    save_steps=500,  # Frequência de salvamento
    evaluation_strategy="no",  # Sem avaliação durante treino
    save_total_limit=2,  # Máximo de checkpoints salvos
    remove_unused_columns=False,  # Manter colunas não usadas
    dataloader_pin_memory=False,  # Otimização de memória
    report_to=None,  # Sem relatórios externos
)

print("📋 Configuração de treinamento:")
print(f"   Épocas: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Warmup steps: {training_args.warmup_steps}")
print(f"   Output dir: {training_args.output_dir}")

print("\n💡 Estes parâmetros são otimizados para treinamento rápido e eficiente!")

## **5. Iniciando o treinamento**

Agora vamos treinar nosso modelo:

In [ ]:
# Iniciando treinamento
print("🚀 INICIANDO TREINAMENTO\n")

# Criando trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_hf,
    tokenizer=tokenizer,
)

print("✅ Trainer criado!")
print("\n🎯 Iniciando treinamento...")
print("⏱️  Isso pode levar alguns minutos...")

# Treinamento (comentado para demonstração)
try:
    # Descomente a linha abaixo para treinar de verdade
    # trainer.train()
    print("\n⚠️  Treinamento comentado para demonstração")
    print("💡 Para treinar de verdade, descomente a linha trainer.train()")
    
except Exception as e:
    print(f"\n❌ Erro no treinamento: {e}")

print("\n✅ Configuração de treinamento concluída!")

## **6. Monitoramento e métricas**

Vamos criar funções para monitorar o treinamento:

In [ ]:
# Funções de monitoramento
def plot_training_metrics(losses, learning_rates):
    """
    Plota métricas de treinamento
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot de loss
    ax1.plot(losses, 'b-', linewidth=2)
    ax1.set_title('📉 Loss de Treinamento')
    ax1.set_xlabel('Steps')
    ax1.set_ylabel('Loss')
    ax1.grid(True, alpha=0.3)
    
    # Plot de learning rate
    ax2.plot(learning_rates, 'r-', linewidth=2)
    ax2.set_title('📈 Learning Rate')
    ax2.set_xlabel('Steps')
    ax2.set_ylabel('Learning Rate')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Simulando métricas de treinamento
print("📊 SIMULANDO MÉTRICAS DE TREINAMENTO\n")

steps = list(range(0, 100, 10))
losses = [2.5, 2.1, 1.8, 1.5, 1.3, 1.1, 0.9, 0.8, 0.7, 0.6]  # Loss diminuindo
learning_rates = [0.0002, 0.00018, 0.00016, 0.00014, 0.00012, 0.0001, 0.00008, 0.00006, 0.00004, 0.00002]  # LR diminuindo

print("📈 Métricas simuladas:")
print(f"   Loss inicial: {losses[0]:.2f}")
print(f"   Loss final: {losses[-1]:.2f}")
print(f"   Melhoria: {((losses[0] - losses[-1]) / losses[0] * 100):.1f}%")

# Plotando métricas
plot_training_metrics(losses, learning_rates)

print("💡 Loss diminuindo = modelo aprendendo! Learning rate diminuindo = convergência!")

## **7. Salvando o modelo treinado**

Vamos salvar nosso modelo treinado:

In [ ]:
# Salvando modelo treinado
print("💾 SALVANDO MODELO TREINADO\n")

def salvar_modelo(model, tokenizer, output_dir):
    """
    Salva o modelo treinado
    """
    try:
        # Salvando modelo LoRA
        model.save_pretrained(output_dir)
        
        # Salvando tokenizer
        tokenizer.save_pretrained(output_dir)
        
        print(f"✅ Modelo salvo em: {output_dir}")
        
        # Salvando configuração
        config_info = {
            "model_name": "microsoft/DialoGPT-small",
            "training_epochs": 3,
            "learning_rate": 2e-4,
            "dataset_size": len(dataset_vendas),
            "lora_config": {
                "r": 16,
                "alpha": 32,
                "dropout": 0.1
            }
        }
        
        with open(f"{output_dir}/config.json", 'w') as f:
            json.dump(config_info, f, indent=2)
        
        print(f"✅ Configuração salva em: {output_dir}/config.json")
        
    except Exception as e:
        print(f"❌ Erro ao salvar: {e}")

# Salvando modelo
output_dir = "./models/iphone_assistant"
salvar_modelo(model, tokenizer, output_dir)

print("\n🎉 Modelo treinado e salvo com sucesso!")

## **8. Teste Rápido**

Vamos testar seu entendimento sobre treinamento:

In [ ]:
# Teste rápido sobre treinamento
print("🧪 TESTE RÁPIDO - TREINAMENTO\n")

perguntas_teste = [
    {
        "pergunta": "Qual é o objetivo do LoRA no fine tuning?",
        "opcoes": [
            "A) Aumentar a velocidade", 
            "B) Reduzir o uso de memória", 
            "C) Melhorar a qualidade"
        ],
        "resposta": "B",
        "explicacao": "LoRA reduz o uso de memória treinando apenas uma pequena parte do modelo!"
    },
    {
        "pergunta": "O que significa quando a loss diminui durante o treinamento?",
        "opcoes": [
            "A) O modelo está piorando", 
            "B) O modelo está aprendendo", 
            "C) O modelo está quebrando"
        ],
        "resposta": "B",
        "explicacao": "Loss diminuindo significa que o modelo está aprendendo e melhorando!"
    },
    {
        "pergunta": "Quantas épocas são recomendadas para fine tuning?",
        "opcoes": [
            "A) 1-5 épocas", 
            "B) 10-20 épocas", 
            "C) 50+ épocas"
        ],
        "resposta": "A",
        "explicacao": "1-5 épocas são suficientes para fine tuning, evitando overfitting!"
    }
]

for i, q in enumerate(perguntas_teste, 1):
    print(f"❓ {i}. {q['pergunta']}")
    for opcao in q['opcoes']:
        print(f"   {opcao}")
    print(f"💡 Resposta: {q['resposta']} - {q['explicacao']}")
    print()

print("🎉 Parabéns! Você já entende os fundamentos do treinamento!")

## **🎉 Módulo 4 Concluído!**

### **O que aprendemos:**

✅ **Configuração de LoRA** para treinamento eficiente  
✅ **Preparação de dados** para treinamento  
✅ **Configuração de argumentos** de treinamento  
✅ **Inicialização do treinamento**  
✅ **Monitoramento de métricas**  
✅ **Salvamento do modelo** treinado

### **Próximos Passos:**

🚀 **Módulo 5**: Avaliando como um professor corrige prova  
🚀 **Módulo 6**: Deploy como abrir um restaurante

---

**💡 Dica do Instrutor**: Treinar uma IA é como treinar um atleta - precisa de paciência, monitoramento e ajustes! Agora que treinamos nossa IA, vamos ver se ela aprendeu bem! 😄

**🚀 Próximo módulo**: Vamos avaliar a qualidade do nosso modelo treinado!